In [1]:
import pandas as pd
import py4cytoscape as p4c
import time
import requests

In [2]:
p4c.__version__

'1.6.0'

In [3]:
p4c.cytoscape_ping()

You are connected to Cytoscape!


'You are connected to Cytoscape!'

In [52]:
def create3Dnetwork(networkLayers, stylexml):
    layerTable = pd.read_table(networkLayers, header=None, sep="\t")
    networkSUID = layerTable.apply(importLayer, axis=1)
    layerTable = pd.concat([layerTable, networkSUID], axis=1)
    nodetables = layerTable.apply(getNodeTableWithLayerinfo, axis=1)
    layeredNodes = getLayeredNodes(nodetables)
    edgetables = layerTable.apply(getEdgeTableWithLayerinfo, axis=1)
    layeredEdges = pd.concat(edgetables.tolist())
    # layeredNodes.to_csv("layeredNodes.csv")
    # layeredEdges.to_csv("layeredEdges.csv")
    p4c.commands_post('cy3d set renderer')
    suID = p4c.create_network_from_data_frames(layeredNodes.astype(str), layeredEdges.astype(str))
    setTransomicStyle(stylexml, suID)
    return suID

In [47]:
def importLayer(row: pd.Series) -> int:
    #print(row.array)
    #getKgml(row.array[1])
    r = requests.get("https://rest.kegg.jp/get/" + row.array[1] + "/kgml")
    kgml = row.array[1] + ".xml"
    f = open(kgml, "w")
    f.write(r.text)
    f.close()    
    print("Importing " + kgml)
    res = p4c.import_network_from_file(kgml)
    time.sleep(3)
    xy = p4c.get_table_columns(table='node', columns=['KEGG_NODE_X', 'KEGG_NODE_Y'])
    xy = xy.rename(columns={"KEGG_NODE_X": "x_location", "KEGG_NODE_Y": "y_location"})
    p4c.load_table_data(xy, table_key_column="SUID")
    return res['networks'][0]

In [6]:
def getNodeTableWithLayerinfo(row: pd.Series) -> pd.DataFrame:
    nt = p4c.get_table_columns(table="node", network=row.array[4])
    nt['z_location'] = row.array[2]
    nt['LAYER_INDEX'] = row.array[0]
    return nt

In [7]:
def getLayeredNodes(nodetables: pd.DataFrame) -> pd.DataFrame:
    nodetable3d = pd.concat(nodetables.array)
    return nodetable3d.rename(columns={"SUID": "id"})

In [8]:
def getEdgeTableWithLayerinfo(row: pd.Series) -> pd.DataFrame:
    et = p4c.get_table_columns(table="edge", network=row.array[4])
    #return et
    print("Getting edge info. This function is kinda slow...")
    ei = p4c.get_edge_info(et["SUID"].tolist(), network=row.array[4])
    print("Finished getting edge info.")
    et['source'] = list(map(lambda x: x['source'], ei))
    et['target'] = list(map(lambda x: x['target'], ei))
    et['LAYER_INDEX'] = row.array[0]
    return et

In [39]:
def setTransomicStyle(xml, suid) -> None:
    #xml is must be a local file path
    stylename = p4c.import_visual_styles(xml)
    res = p4c.set_visual_style(stylename[0], network = suid)
    print(stylename[0] + " " + res['message'])

In [53]:
suid = create3Dnetwork("https://raw.githubusercontent.com/ecell/transomics2cytoscape/master/inst/extdata/usecase1/yugi2014.tsv", "yugi2014.xml")

Importing rno04910.xml
Importing rno01100.xml
Importing rno01100.xml
Column "KEGG_NODE_LABEL_LIST" has only 73 elements, but should have 74
Getting edge info. This function is kinda slow...
Finished getting edge info.
Column "KEGG_EDGE_SUBTYPES" has only 0 elements, but should have 1312
Getting edge info. This function is kinda slow...
Finished getting edge info.
Column "KEGG_EDGE_SUBTYPES" has only 0 elements, but should have 1312
Getting edge info. This function is kinda slow...
Finished getting edge info.
Applying default style...
Applying preferred layout
yugi2014 Visual Style applied.
